In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('./../DATASETS/augmented_movies_reviews.csv')
df.head()

Reviews  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           निर्माता:\nशीतल विनोद तलवार, मधु मन्न्टेना\n\nनिदेशक :\nरामगोपाल वर्मा\n\nकलाकार :\nविवेक ओबेरोई, सौत्रुघन सिन्हा, अभिमन्यु सिंह, सुशज़िंग सिंह, ज़रीन वहाब, आशीष विद्यार्थी, राजा कृष्णमूर्ति, श्रीनिवास राव\n\n* वयस्कों के लिए केवल * 16 कॉइल * 2 घंटे 14 मिनट\n\n\nजैसे ही "Raktcharitra-1" शुरू होता है, स्पष्टीकरण लिखा जाता है कि सभी पात्र और कहानी निम्नलिखित हैं। जटिलताएं।\n\n"रकटापारित्रा" की कहानी आंध्र प्रदेश और उनके सूरी द्वारा रवि परित्ता पर आधारित है।\n\nपहले भाग से पता चलता है कि (फिल्म में प्रताप) अपने पिता और उसके भाई की हत्या के बाद बदला लेता है। शामिल।\n\nजब प्रताप ने तीनों को मार डाला, तो नागमानी का बेटा, बुका (अभिमनु सिंह), उस पर बदला लेने के लिए अधीरता है। एक क्रूर विधि के साथ (एक गन्ने के साथ सिर को काटें, आरी मशीन से छाती को अलग करना, आदि)।\n\nPratap चुनाव जीतता है और एक अंधविश्वासी और एक राजनेता (वास्तव में nt राम राव) की मदद से बुक्का के भाई के खिलाफ बुका को समाप्त करता है। सूरी नामक पथ।\n\nफिल्म उन लोगों के लिए दिलचस्प हो सकती है जो आंध्र प्रदेश की नीति को बारीकी से जानते हैं, लेकिन जो लोग उन्हें नहीं जानते हैं, वे इस नियमित कहानी को पाते हैं, जिसम

In [3]:
# Install required libraries
# !pip install transformers torch pandas scikit-learn indic-nlp-library

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

# Load your dataset (ensure it's a DataFrame with columns 'text' and 'label')
data = df.copy()  # Replace with your dataset path

# Check data format
# assert 'text' in data.columns and 'label' in data.columns, "Dataset must have 'text' and 'label' columns"

texts = data['Reviews'].tolist()
labels = data['labels'].tolist()

# Map labels to integers if not already done
label_mapping = {label: idx for idx, label in enumerate(set(labels))}
data['labels'] = data['labels'].map(label_mapping)
labels = data['labels'].tolist()

# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)


model_name = 'microsoft/deberta-v3-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Tokenize the datasets
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

train_encodings = tokenize_function(train_texts)
test_encodings = tokenize_function(test_texts)

# Convert labels to PyTorch tensors
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Check if GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Create a Dataset class
class HindiTextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

# Prepare datasets
train_dataset = HindiTextDataset(train_encodings, train_labels)
test_dataset = HindiTextDataset(test_encodings, test_labels)

# Define evaluation metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=50,
    weight_decay=0.01,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=3,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=7)]
)

# Train the model
trainer.train()

# Evaluate the model on the test dataset
test_results = trainer.evaluate(test_dataset)
print("Test Results:", test_results)

# Predict on new Hindi text
def classify_text(text):
    tokens = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(**tokens)
    prediction = torch.argmax(outputs.logits, dim=1)
    return list(label_mapping.keys())[prediction.item()]  # Return the original label

# Test the model on a sample
sample_text = "यह एक परीक्षण वाक्य है।"  # Replace with your Hindi text
predicted_label = classify_text(sample_text)
print(f"Predicted Label for '{sample_text}': {predicted_label}")


/home/lenovo/.local/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/lenovo/.local/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will 

  0%|          | 0/37200 [00:00<?, ?it/s]

{'loss': 1.0517, 'grad_norm': 5.317065238952637, 'learning_rate': 1.9731182795698928e-05, 'epoch': 0.67}


  0%|          | 0/186 [00:00<?, ?it/s]

{'eval_loss': 1.0666142702102661, 'eval_accuracy': 0.4422043010752688, 'eval_f1': 0.3739032903601039, 'eval_precision': 0.4580926419928621, 'eval_recall': 0.4422043010752688, 'eval_runtime': 26.7877, 'eval_samples_per_second': 55.548, 'eval_steps_per_second': 6.943, 'epoch': 1.0}
{'loss': 1.0009, 'grad_norm': 2.8891122341156006, 'learning_rate': 1.946236559139785e-05, 'epoch': 1.34}


...

{'eval_loss': 1.6881157159805298, 'eval_accuracy': 0.8366935483870968, 'eval_f1': 0.837694083790657, 'eval_precision': 0.8403879202320004, 'eval_recall': 0.8366935483870968, 'eval_runtime': 15.8509, 'eval_samples_per_second': 93.875, 'eval_steps_per_second': 11.734, 'epoch': 37.0}
{'loss': 0.0136, 'grad_norm': 0.00016080585191957653, 'learning_rate': 4.946236559139785e-06, 'epoch': 37.63}


  0%|          | 0/186 [00:00<?, ?it/s]

{'eval_loss': 1.6843794584274292, 'eval_accuracy': 0.844758064516129, 'eval_f1': 0.8451253013624048, 'eval_precision': 0.8460115026571617, 'eval_recall': 0.844758064516129, 'eval_runtime': 15.7868, 'eval_samples_per_second': 94.256, 'eval_steps_per_second': 11.782, 'epoch': 38.0}
{'train_runtime': 11641.958, 'train_samples_per_second': 25.563, 'train_steps_per_second': 3.195, 'train_loss': 0.23182206162386987, 'epoch': 38.0}


  0%|          | 0/186 [00:00<?, ?it/s]

Test Results: {'eval_loss': 1.5190038681030273, 'eval_accuracy': 0.8454301075268817, 'eval_f1': 0.8451949511547825, 'eval_precision': 0.847356804309445, 'eval_recall': 0.8454301075268817, 'eval_runtime': 15.0775, 'eval_samples_per_second': 98.69, 'eval_steps_per_second': 12.336, 'epoch': 38.0}
Predicted Label for 'यह एक परीक्षण वाक्य है।': neutral
